In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

In [2]:
import requests
import time
import json

BASE_URL = "https://pncp.gov.br/api/consulta/v1/"

# Define parameters for the API request
params = {
    "dataInicial": "20230101",  # Example format: AAAAMMDD
    "dataFinal": "20231231",    # Example format: AAAAMMDD
    "codigoModalidadeContratacao": 8,  # Example: "8" corresponds to Dispensa de Licitação
    "pagina": 10
}

# Initialize a list to store the data
todos_os_dados = []

# Function to handle API requests with retries and debug information
def fetch_data(params, retries=3):
    for attempt in range(retries):
        try:
            print(f"Requesting page {params['pagina']} with parameters: {params}")
            response = requests.get(BASE_URL + "contratacoes/publicacao", params=params)
            
            if response.status_code == 200:
                return response.json()  # Return JSON if successful
            else:
                print(f"Error: {response.status_code}. Response: {response.text}. Attempt {attempt+1}/{retries}")
        except requests.RequestException as e:
            print(f"Request error: {e}. Attempt {attempt+1}/{retries}")
        time.sleep(1)  # Wait before retrying
    return None  # Return None if all retries fail

# Function to save partial results to a file
def save_partial_results(data, filename="partial_results.json"):
    with open(filename, "a") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.write("\n")

# First API call
result = fetch_data(params)

if result:
    todos_os_dados.extend(result.get('data', []))  # Collect the first page of data
    total_paginas = result.get('totalPaginas', 1)  # Total pages of results
    
    # Save the first page's data
    save_partial_results(result.get('data', []))

    # Loop through remaining pages
    for pagina in range(2, total_paginas + 1):
        params["pagina"] = pagina  # Update the page number
        
        result = fetch_data(params)
        if result:
            todos_os_dados.extend(result.get('data', []))
            
            # Save each page's data incrementally
            save_partial_results(result.get('data', []))
        else:
            print(f"Failed to retrieve data for page {pagina}")

    print(f"Total records collected: {len(todos_os_dados)}")
else:
    print("Failed to retrieve any data.")


Requesting page 10 with parameters: {'dataInicial': '20230101', 'dataFinal': '20231231', 'codigoModalidadeContratacao': 8, 'pagina': 10}
Requesting page 2 with parameters: {'dataInicial': '20230101', 'dataFinal': '20231231', 'codigoModalidadeContratacao': 8, 'pagina': 2}
Requesting page 3 with parameters: {'dataInicial': '20230101', 'dataFinal': '20231231', 'codigoModalidadeContratacao': 8, 'pagina': 3}
Requesting page 4 with parameters: {'dataInicial': '20230101', 'dataFinal': '20231231', 'codigoModalidadeContratacao': 8, 'pagina': 4}
Requesting page 5 with parameters: {'dataInicial': '20230101', 'dataFinal': '20231231', 'codigoModalidadeContratacao': 8, 'pagina': 5}
Requesting page 6 with parameters: {'dataInicial': '20230101', 'dataFinal': '20231231', 'codigoModalidadeContratacao': 8, 'pagina': 6}
Requesting page 7 with parameters: {'dataInicial': '20230101', 'dataFinal': '20231231', 'codigoModalidadeContratacao': 8, 'pagina': 7}
Requesting page 8 with parameters: {'dataInicial': '2

UnicodeEncodeError: 'charmap' codec can't encode characters in position 171-172: character maps to <undefined>